# Setup

## Mount Filesystem

In [ ]:
# Set up colab filesystem
import os
import sys
os.environ['NOTEBOOK_MODE'] = '1'
from google.colab import drive

os.chdir('/content')
drive.mount('drive')
WORKDIR = 'drive/MyDrive/Colab Notebooks/ECE661/Project' # Change to location in drive
sys.path.append(WORKDIR)
os.chdir(WORKDIR)

## Dependencies

In [ ]:
# Install python dependencies
!pip install -r requirements.txt

In [ ]:
# Import dependencies
import torch as ch
from torch.nn.functional import interpolate
from torch.distributions.multivariate_normal import MultivariateNormal
import numpy as np
import seaborn as sns
from scipy import stats
from skimage.transform import resize
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
from robustness import model_utils, datasets
from robustness.tools.vis_tools import show_image_row, show_image_column
from robustness.tools.label_maps import CLASS_DICT
from user_constants import DATA_PATH_DICT
%matplotlib inline

## Constants

In [ ]:
# Constants
DATA = 'ImageNet' # Choices: ['CIFAR', 'ImageNet'] Use ImageNet for ImageNette
IMAGENETTE_CLASSES = [0, 217, 482, 491, 497, 566, 569, 571, 574, 701] # Map index to target class
IMAGENETTE_CLASS_NAMES = ['tench', 'english springer', 'casette player', 
                          'chain saw', 'church', 'french horn', 
                          'garbage truck', 'gas pump', 'golf ball', 'parachute']
BATCH_SIZE = 10
NUM_WORKERS = 2
NUM_CLASSES_VIS = 10
DEVICE = 'cuda'

DATA_SHAPE = 32 if DATA == 'CIFAR' else 224 # Image size (fixed for dataset)
REPRESENTATION_SIZE = 2048 # Size of representation vector (fixed for model)
CLASSES = CLASS_DICT[DATA] # Class names for dataset
NUM_CLASSES = len(CLASSES) - 1 
NUM_CLASSES_VIS = min(NUM_CLASSES_VIS, NUM_CLASSES)
GRAIN = 4 if DATA != 'CIFAR' else 1
CLASS_IDS = IMAGENETTE_CLASSES if DATA == 'ImageNet' else range(NUM_CLASSES_VIS)

## Load Dataset and Pretrained Model

In [ ]:
# Load dataset
dataset_function = getattr(datasets, DATA)
dataset = dataset_function(DATA_PATH_DICT[DATA])
train_loader, test_loader = dataset.make_loaders(workers=NUM_WORKERS, 
                                      batch_size=BATCH_SIZE, 
                                      data_aug=False)
data_iterator = enumerate(test_loader)

In [ ]:
# Load model
model_kwargs = {
    'arch': 'resnet50',
    'dataset': dataset,
    'resume_path': f'./models/{DATA}.pt'
}

model, _ = model_utils.make_and_restore_model(**model_kwargs)
model.eval()

In [ ]:
def downsample(x, step=GRAIN):
    down = ch.zeros([len(x), 3, DATA_SHAPE//step, DATA_SHAPE//step])

    for i in range(0, DATA_SHAPE, step):
        for j in range(0, DATA_SHAPE, step):
            v = x[:, :, i:i+step, j:j+step].mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            ii, jj = i // step, j // step
            down[:, :, ii:ii+1, jj:jj+1] = v
    return down

def upsample(x, step=GRAIN):
    up = ch.zeros([len(x), 3, DATA_SHAPE, DATA_SHAPE])

    for i in range(0, DATA_SHAPE, step):
        for j in range(0, DATA_SHAPE, step):
            ii, jj = i // step, j // step
            up[:, :, i:i+step, j:j+step] = x[:, :, ii:ii+1, jj:jj+1]
    return up

In [ ]:
def index_to_class(targ):
    # Convert index as read from dataloader to actual target class
    targ_list = []
    for i in targ:
        targ_list.append(CLASS_IDS[i])
    return ch.tensor(targ_list)

# Image Generation

## Calculate Random Seeds

In [ ]:
# Get seed distribution (can be memory intensive to do all ImageNet classes at once)

im_test, targ_test = [], []
for _, (im, targ) in enumerate(test_loader):
    im_test.append(im)
    targ_test.append(index_to_class(targ))
im_test, targ_test = ch.cat(im_test), ch.cat(targ_test)


conditionals = {}
for i in tqdm(CLASS_IDS):
    imc = im_test[targ_test == i]
    down_flat = downsample(imc).view(len(imc), -1)
    mean = down_flat.mean(dim=0)
    down_flat = down_flat - mean.unsqueeze(dim=0)
    cov = down_flat.t() @ down_flat / len(imc)
    dist = MultivariateNormal(mean, covariance_matrix=cov+1e-4*ch.eye(3 * DATA_SHAPE//GRAIN * DATA_SHAPE//GRAIN))
    conditionals[i] = dist

In [ ]:
# Visualize seeds
img_seed = ch.stack([conditionals[i].sample().view(3, DATA_SHAPE//GRAIN, DATA_SHAPE//GRAIN) 
                     for i in CLASS_IDS])
img_seed = ch.clamp(img_seed, min=0, max=1)
show_image_row([img_seed.cpu()], tlist=[[f'Class {i}' for i in CLASS_IDS]])

## Generate Images

In [ ]:
def generation_loss(mod, inp, targ):
    op = mod(inp)
    loss = ch.nn.CrossEntropyLoss(reduction='none')(op, targ)
    return loss, None

kwargs = {
        'custom_loss': generation_loss,
        'constraint':'2',
        'eps': 40,
        'step_size': 1,
        'iterations': 60,
        'targeted': True,
}  
    
if DATA == 'CIFAR':
    kwargs['eps'] = 30
    kwargs['step_size'] = 0.5
    kwargs['iterations'] = 60

In [ ]:
show_seed = False
for i in CLASS_IDS:
    target_class = i * ch.ones((BATCH_SIZE, ))
    im_seed = ch.stack([conditionals[int(t)].sample().view(3, DATA_SHAPE//GRAIN, DATA_SHAPE//GRAIN) 
                        for t in target_class])
    
    im_seed = upsample(ch.clamp(im_seed, min=0, max=1)).to(DEVICE)
    _, im_gen = model(im_seed, target_class.long().to(DEVICE), make_adv=True, **kwargs)
    if show_seed:
        show_image_row([im_seed.cpu()], [f'Seed ($x_0$)'], fontsize=18)
    show_image_row([im_gen.detach().cpu()], 
                   [CLASSES[int(t)].split(',')[0] for t in target_class], 
                   fontsize=18)

# Inpainting

## Image Corruption

In [ ]:
PATCH_SIZE = 6 if DATA == "CIFAR" else 60

# Function to get corrupted image
def mask_image(x, width=PATCH_SIZE):
    loc = np.random.randint(0, x.shape[-1] - PATCH_SIZE, size=(x.shape[0], 2))
    mask = ch.zeros_like(x)
    for idx in range(x.shape[0]):
        i, j = loc[idx, 0], loc[idx, 1]
        val = ch.mean(ch.mean(x[idx, :], dim=2, keepdim=True), dim=1, keepdim=True)
        # Initialize masked region as mean pixel value over image (per channel)
        x[idx, :, i:i+PATCH_SIZE, j:j+PATCH_SIZE] = val.expand_as(x[idx, :, i:i+PATCH_SIZE, j:j+PATCH_SIZE])
        mask[idx, :, i:i+PATCH_SIZE, j:j+PATCH_SIZE] = 1
    return x, mask

## Inpainting

In [ ]:
# Custom inpainting loss
def inpainting_loss_wrapper(im_targ, mask, normalizer, lambdar=10):
    def inpainting_loss(mod, inp, targ):
        op = mod(normalizer(inp), fake_relu=True)
        loss = ch.nn.CrossEntropyLoss(reduction='none')(op, targ)
        loss_l2 = ((im_targ - inp) * (1 - mask) )**2
        loss_l2 = loss_l2.mean(-1).mean(-1).mean(-1)
        loss += lambdar * loss_l2
        return loss, None
    return inpainting_loss

In [ ]:
# PGD parameters
kwargs = {
        'constraint':'2',
        'eps': 21.6,
        'step_size': 0.1,
        'iterations': 720, 
        'do_tqdm': True,
        'targeted': True,
        'should_normalize': False
}

In [ ]:
# Inpainting using PGD
_, (img_orig, targ_orig) = next(data_iterator)
targ_orig = index_to_class(targ_orig)

img_corrupt, mask = mask_image(img_orig.clone())
img_corrupt = img_corrupt.to(DEVICE)

kwargs['custom_loss'] = inpainting_loss_wrapper(img_corrupt.cuda(), 
                                                mask.cuda(), 
                                                model.normalizer)

_, img_inpaint = model(img_corrupt, targ_orig.clone().to(DEVICE), make_adv=True, **kwargs)

In [ ]:
# Visualize inpainted images
show_image_row([img_corrupt.cpu(), img_orig.cpu(), img_inpaint.detach().cpu()], 
               ["Corrupted", "Original", "Inpainted"],
               fontsize=22)

# Super-Resolution

## Generate Downsampled Images

In [ ]:
# Get images and classes
DOWNSAMPLED_SIZE = (16, 16) if DATA == 'CIFAR' else (64, 64)
ORIGINAL_SIZE = (32, 32) if DATA == 'CIFAR' else (224, 224)
STEP = 2 if DATA == 'CIFAR' else 7

(im, targ) = next(iter(train_loader))
img = im.clone().detach().to(DEVICE)
targ = index_to_class(targ).clone().detach().to(DEVICE)
img_downsampled = interpolate(img, size=DOWNSAMPLED_SIZE, mode='nearest').to(DEVICE)

## Bicubic Upsampling

In [ ]:
img_bicubic = interpolate(img_downsampled, size=ORIGINAL_SIZE, mode='bicubic').clip(0, 1)

## Super-Resolution Upsampling

In [ ]:
# PGD Parameters
kwargs = {
        'constraint':'2',
        'eps': 15,
        'step_size': 1,
        'iterations': 50,
        'do_tqdm':True,
        'targeted': True,
        'should_normalize': True
}

In [ ]:
# Superresolution using PGD
#img_up = upsample(img_downsampled, step=STEP).to(DEVICE)
img_up = interpolate(img_downsampled, size=ORIGINAL_SIZE, mode='nearest').clip(0, 1)
_, img_sr = model(img_up, targ, fake_relu=False, make_adv=True, **kwargs)

In [ ]:
# Visualize superres results
print(img_sr.shape)
show_image_row([img.cpu(), img_downsampled.cpu(), img_bicubic.cpu(), img_sr.detach().cpu()], 
               [f'Original\n{ORIGINAL_SIZE}', f'Downsampled\n{DOWNSAMPLED_SIZE}', 
                f'Bicubic\n{ORIGINAL_SIZE}', f'{STEP}x Superres\n{ORIGINAL_SIZE}'],
               fontsize=18)